In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_path = "/content/drive/MyDrive/J_comp_Neural_network/dataset_1/data/train"
valid_path = "/content/drive/MyDrive/J_comp_Neural_network/dataset_1/data/val"

In [ ]:
IMAGE_SIZE = [256, 256]

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in inception.layers:
  layer.trainable = False

87924736/87910968 [==============================] - 2s 0us/step


In [ ]:
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
x = Flatten()(inception.output)
x = BatchNormalization()(x)
x = Dense(32,activation = 'relu')(x)
x = Dense(64,activation = 'relu')(x)

In [ ]:
x = Flatten()(vgg.output)

In [ ]:
prediction = Dense(5, activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
model_inception = Model(inputs=inception.input, outputs=prediction)

In [ ]:
model_inception.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
import tensorflow as tf

In [ ]:
model_inception.compile(
  loss='categorical_crossentropy',
  optimizer= tf.keras.optimizers.Adam(
    learning_rate=0.001,
),

  metrics=['accuracy']
)

In [ ]:
train_d = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.5, 1.5],
)

train_generator = train_d.flow_from_directory(
        '/content/drive/MyDrive/J_comp_Neural_network/dataset_1/data/train',
        target_size=(256,256),
        batch_size=32,
        class_mode="categorical",

        #save_to_dir = "/content/drive/MyDrive/J_comp_Neural_network/dataset/augmented_images"

)

Found 4522 images belonging to 5 classes.


In [ ]:
valid_d = ImageDataGenerator(
        rescale = 1./255,
        brightness_range=[0.5, 1.5],

)

valid_generator = valid_d.flow_from_directory(
        '/content/drive/MyDrive/J_comp_Neural_network/dataset_1/data/val',
        target_size=(256,256),
        batch_size=32,
        class_mode="categorical",
        
)

Found 563 images belonging to 5 classes.


## **VGG 16 Transfer Learning + data augmented**

In [ ]:
r = model.fit_generator(
  train_generator,
  validation_data=test_generator,
  epochs=20,
  steps_per_epoch=100,
  validation_steps=40
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/20
100/100 [==============================] - 91s 910ms/step - loss: 0.9987 - accuracy: 0.6662 - val_loss: 1.0805 - val_accuracy: 0.6562
Epoch 2/20
100/100 [==============================] - 90s 900ms/step - loss: 0.9930 - accuracy: 0.6658 - val_loss: 1.2660 - val_accuracy: 0.6172
Epoch 3/20
100/100 [==============================] - 91s 908ms/step - loss: 1.0720 - accuracy: 0.6712 - val_loss: 0.9777 - val_accuracy: 0.6849
Epoch 4/20
100/100 [==============================] - 90s 901ms/step - loss: 0.9561 - accuracy: 0.6871 - val_loss: 1.3921 - val_accuracy: 0.6536
Epoch 5/20
100/100 [==============================] - 90s 896ms/step - loss: 0.9942 - accuracy: 0.6724 - val_loss: 1.0380 - val_accuracy: 0.6458
Epoch 6/20
100/100 [==============================] - 89s 891ms/step - loss: 1.1358 - accuracy: 0.6569 - val_loss: 1.0363 - val_accuracy: 0.6875
Epoch 7/20
100/100 [==============================] - 89s 889ms/step - loss: 1.1177 - accuracy: 0.6558 - val_loss: 1.2291 - val_ac

# **RES NET**

In [ ]:
r1 = model_inception.fit(
  train_generator,
  validation_data=valid_generator,
  epochs=20,
  steps_per_epoch=49,
  validation_steps=12
)

Epoch 1/20
49/49 [==============================] - 678s 14s/step - loss: 2.6150 - accuracy: 0.5310 - val_loss: 1.9341 - val_accuracy: 0.4844
Epoch 2/20
49/49 [==============================] - 412s 8s/step - loss: 1.8476 - accuracy: 0.6193 - val_loss: 1.5038 - val_accuracy: 0.6042
Epoch 3/20
49/49 [==============================] - 308s 6s/step - loss: 1.4196 - accuracy: 0.6818 - val_loss: 1.5673 - val_accuracy: 0.5964
Epoch 4/20
49/49 [==============================] - 278s 6s/step - loss: 1.1690 - accuracy: 0.7109 - val_loss: 1.5804 - val_accuracy: 0.5833
Epoch 5/20
49/49 [==============================] - 285s 6s/step - loss: 0.9792 - accuracy: 0.7436 - val_loss: 1.6089 - val_accuracy: 0.5859
Epoch 6/20
49/49 [==============================] - 275s 6s/step - loss: 1.0169 - accuracy: 0.7468 - val_loss: 1.5965 - val_accuracy: 0.5990
Epoch 7/20
49/49 [==============================] - 279s 6s/step - loss: 0.7986 - accuracy: 0.7800 - val_loss: 1.7453 - val_accuracy: 0.5911
Epoch 8/20
4